In [331]:
from __future__ import print_function
import pandas as pd
import requests
from requests_oauthlib import OAuth1
import cnfg
import os

In [332]:
config = cnfg.load(".twitter_config")

oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

In [487]:
harvey_tweets = []
with open("ISCRAM18_datasets/Harvey_tweet_ids.txt") as myfile:
    for line in myfile:
        line = line.strip()
        harvey_tweets.append(line)

In [488]:
len(harvey_tweets)

6506304

# Pull Harvey data subset
- Get tweet jsons
- create dictionary w/ desired features
- add to mongo db & crate df

In [489]:
import json
from pymongo import MongoClient

client = MongoClient()
db = client.twitter
tweets_collection = db.tweets

In [510]:
def get_tweet_metadata(tweet,meta_dict):
    meta_dict['storm'] = 'Harvey'
    try: meta_dict['id']  = tweet['id']
    except: pass
    try: meta_dict['user_id']  = tweet['entities']['user_mentions'][0]['id']
    except: pass
    try: meta_dict['user_name']  = tweet['entities']['user_mentions'][0]['name']
    except: pass
    try: meta_dict['datetime'] = tweet['created_at']
    except: pass
    try: meta_dict['media'] = tweet['entities']['media'][0]['media_url']
    except: pass
    try: meta_dict['media_type'] = tweet['extended_entities']['media'][0]['type']
    except: pass
    try: meta_dict['coords'] = tweet['geo']['coordinates']
    except: pass
    try: meta_dict['text'] = tweet['text']
    except: pass
    try: 
        hashtags_text = tweet['entities']['hashtags']
        hashtags = []
        for i in hashtags_text:
            hashtags.append(i['text'])
        meta_dict['hashtags'] = hashtags
    except: pass
    try: meta_dict['followers'] = tweet['extended_entities']['media'][0]['additional_media_info']['source_user']['followers_count']
    except: pass
    try: meta_dict['following'] = tweet['extended_entities']['media'][0]['additional_media_info']['source_user']['friends_count']
    except: pass
    try: meta_dict['user_location'] = tweet['extended_entities']['media'][0]['additional_media_info']['source_user']['location']
    except: pass
    try: meta_dict['retweets'] = tweet['retweet_count']
    except: pass

In [506]:
for tweet_id in harvey_tweets:
    meta_dict = {}
    parameters = {"id": int(tweet_id)}
    response = requests.get("https://api.twitter.com/1.1/statuses/show.json",
                            params = parameters,
                            auth=oauth)
    tweet = response.json()
    get_tweet_metadata(tweet,meta_dict)
    if len(meta_dict)>0:
        tweets_collection.insert_one(meta_dict)
    

In [507]:
tweets_dict = []
for tweet in tweets_collection.find():
    tweets_dict.append(tweet)

In [509]:
df = pd.DataFrame(tweets_dict)
df

_id                       coords  \
0  5b0613d6a54d750dbd5579d1  [29.73824392, -95.49140669]   
1  5b0613d6a54d750dbd5579d2                          NaN   
2  5b0613d6a54d750dbd5579d3                          NaN   
3  5b0613d6a54d750dbd5579d4                          NaN   

                         datetime               hashtags                  id  \
0  Wed Aug 30 23:34:42 +0000 2017      [Houston, Harvey]  903038298880229376   
1  Sun Aug 27 03:22:22 +0000 2017               [Harvey]  901646039555256320   
2  Sun Aug 27 03:22:22 +0000 2017                     []  901646039886708736   
3  Sun Aug 27 03:22:22 +0000 2017  [Harvey, houwx, txwx]  901646039957962753   

                                            media media_type  retweets  \
0                                             NaN        NaN         5   
1  http://pbs.twimg.com/media/DING35qXcAEgLUX.jpg      photo         7   
2                                             NaN        NaN        22   
3  http://pbs.twimg.com/media/DIL5txPV4AAY2Yu.jpg      photo       597   

                                                text       user_id  \
0  Very nice thing to do for those that risk thei...           NaN   
1  RT @scribemark: 1 day in and #Harvey proving t...  1.569257e+07   
2  RT @earthlydevon: Bro like wtf you want me to ...  7.352977e+17   
3  RT @fema: Texas: If you're under a tornado war...  1.666908e+07   

     user_name  
0          NaN  
1  Mark Hollis  
2            🌍  
3         FEMA

In [503]:
df[df['_id'] == 903038298880229376]

Empty DataFrame
Columns: [_id, coords, datetime, followers, following, hashtags, id, media, media_type, retweets, text, user_id, user_location, user_name]
Index: []

In [475]:
# parameters = {"geocode":'37.781157,-122.398720,1mi'}
# response = requests.get("https://api.twitter.com/1.1/search/tweets.json",
#                         params = parameters,
#                         auth=oauth)
# print(response.json())

In [474]:
# parameters = {"id": 903038298880229376}
# response = requests.get("https://api.twitter.com/1.1/statuses/show.json",
#                         params = parameters,
#                         auth=oauth)
# tweet = response.json()
# print(tweet)

In [444]:
import time
from datetime import time
from datetime import datetime
import re

In [439]:
date = df['datetime'][1]
date = re.sub('\+0000 ', '', date)
date

'Sun Aug 27 03:22:22 2017'

In [445]:
datetime.strptime(date, "%a %b %d %X %Y")  

datetime.datetime(2017, 8, 27, 3, 22, 22)

In [505]:
db.drop_collection(tweets_collection)

{'nIndexesWas': 1, 'ns': 'twitter.tweets', 'ok': 1.0}